In [1]:
from PyQt5.QtGui import *
from PyQt5.QtSvg import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

import numpy
import sys
import toyplot.svg
import xml.etree.ElementTree as xml

In [2]:
application = QApplication(sys.argv)

In [3]:
renderer = QSvgRenderer()

In [4]:
numpy.random.seed(1234)
canvas, table = toyplot.matrix(numpy.random.random((10, 10)))
svg = toyplot.svg.render(canvas)

In [5]:
renderer.load(QByteArray(xml.tostring(svg)))

True

In [6]:
image = QImage(renderer.defaultSize(), QImage.Format_ARGB32)
image.fill(QColor("white"))

In [7]:
painter = QPainter(image)

In [8]:
renderer.render(painter)

In [9]:
image.save("test-qt.png")

True